In [ ]:
import React, { useState } from 'react';
import { motion, AnimatePresence } from 'framer-motion';
import { Button } from "@/components/ui/button";
import { Input } from "@/components/ui/input";
import { Badge } from "@/components/ui/badge";
import { 
  ChevronRight, ChevronLeft, Target, Scale, Ruler, 
  Activity, AlertCircle, Sparkles, Check
} from 'lucide-react';

const dietaryGoals = [
  { id: 'weight_loss', label: 'Lose Weight', emoji: '🔥', desc: 'Create a calorie deficit' },
  { id: 'muscle_gain', label: 'Build Muscle', emoji: '💪', desc: 'High protein focus' },
  { id: 'maintenance', label: 'Maintain', emoji: '⚖️', desc: 'Stay balanced' },
  { id: 'health_improvement', label: 'Eat Healthier', emoji: '🥗', desc: 'Improve nutrition' },
];

const dietTypes = [
  { id: 'none', label: 'No Preference', emoji: '🍽️' },
  { id: 'vegan', label: 'Vegan', emoji: '🌱' },
  { id: 'vegetarian', label: 'Vegetarian', emoji: '🥬' },
  { id: 'keto', label: 'Keto', emoji: '🥑' },
  { id: 'paleo', label: 'Paleo', emoji: '🥩' },
  { id: 'mediterranean', label: 'Mediterranean', emoji: '🫒' },
  { id: 'low_carb', label: 'Low Carb', emoji: '🥚' },
  { id: 'diabetic_friendly', label: 'Diabetic Friendly', emoji: '💚' },
];

const activityLevels = [
  { id: 'sedentary', label: 'Sedentary', desc: 'Little to no exercise' },
  { id: 'light', label: 'Light', desc: '1-3 days/week' },
  { id: 'moderate', label: 'Moderate', desc: '3-5 days/week' },
  { id: 'active', label: 'Active', desc: '6-7 days/week' },
  { id: 'very_active', label: 'Very Active', desc: 'Intense daily exercise' },
];

const commonAllergens = ['Peanuts', 'Tree Nuts', 'Milk', 'Eggs', 'Wheat', 'Soy', 'Fish', 'Shellfish', 'Sesame'];

const SlideTransition = ({ children }) => (
  <motion.div
    initial={{ opacity: 0, x: 50 }}
    animate={{ opacity: 1, x: 0 }}
    exit={{ opacity: 0, x: -50 }}
    transition={{ duration: 0.3 }}
  >
    {children}
  </motion.div>
);

export default function OnboardingFlow({ onComplete }) {
  const [step, setStep] = useState(0);
  const [profile, setProfile] = useState({
    dietary_goal: '',
    diet_type: 'none',
    allergies: [],
    weight: '',
    height: '',
    age: '',
    activity_level: 'moderate',
    daily_calorie_target: 2000,
    daily_protein_target: 150,
    daily_carbs_target: 250,
    daily_fats_target: 65,
  });

  const totalSteps = 5;

  const calculateTargets = () => {
    const weight = parseFloat(profile.weight) || 70;
    const height = parseFloat(profile.height) || 170;
    const age = parseFloat(profile.age) || 30;
    
    // Basic BMR calculation (Mifflin-St Jeor)
    let bmr = 10 * weight + 6.25 * height - 5 * age + 5;
    
    const activityMultipliers = {
      sedentary: 1.2,
      light: 1.375,
      moderate: 1.55,
      active: 1.725,
      very_active: 1.9
    };
    
    let tdee = bmr * activityMultipliers[profile.activity_level];
    
    // Adjust based on goal
    if (profile.dietary_goal === 'weight_loss') {
      tdee -= 500;
    } else if (profile.dietary_goal === 'muscle_gain') {
      tdee += 300;
    }
    
    const calories = Math.round(tdee);
    const protein = Math.round(weight * (profile.dietary_goal === 'muscle_gain' ? 2.2 : 1.6));
    const fats = Math.round((calories * 0.25) / 9);
    const carbs = Math.round((calories - (protein * 4) - (fats * 9)) / 4);
    
    return {
      daily_calorie_target: calories,
      daily_protein_target: protein,
      daily_carbs_target: carbs,
      daily_fats_target: fats,
    };
  };

  const handleComplete = () => {
    const targets = calculateTargets();
    onComplete({ 
      ...profile, 
      ...targets,
      onboarding_complete: true 
    });
  };

  const toggleAllergen = (allergen) => {
    setProfile(prev => ({
      ...prev,
      allergies: prev.allergies.includes(allergen)
        ? prev.allergies.filter(a => a !== allergen)
        : [...prev.allergies, allergen]
    }));
  };

  const canProceed = () => {
    switch (step) {
      case 0: return profile.dietary_goal;
      case 1: return true;
      case 2: return profile.weight && profile.height && profile.age;
      case 3: return profile.activity_level;
      case 4: return true;
      default: return true;
    }
  };

  return (
    <div className="min-h-screen bg-gradient-to-br from-slate-50 via-white to-emerald-50/30 flex flex-col">
      {/* Progress */}
      <div className="p-6">
        <div className="flex gap-2">
          {Array.from({ length: totalSteps }, (_, i) => (
            <div 
              key={i}
              className={`h-1.5 flex-1 rounded-full transition-colors ${
                i <= step ? 'bg-emerald-500' : 'bg-slate-200'
              }`}
            />
          ))}
        </div>
      </div>

      {/* Content */}
      <div className="flex-1 px-6 pb-6">
        <AnimatePresence mode="wait">
          {step === 0 && (
            <SlideTransition key="goal">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">What's your goal?</h1>
                  <p className="text-slate-500 mt-1">We'll personalize your experience</p>
                </div>
                <div className="grid gap-3">
                  {dietaryGoals.map(goal => (
                    <motion.button
                      key={goal.id}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile(p => ({ ...p, dietary_goal: goal.id }))}
                      className={`p-4 rounded-2xl border-2 text-left transition-all ${
                        profile.dietary_goal === goal.id
                          ? 'border-emerald-500 bg-emerald-50'
                          : 'border-slate-200 bg-white hover:border-slate-300'
                      }`}
                    >
                      <div className="flex items-center gap-4">
                        <span className="text-3xl">{goal.emoji}</span>
                        <div className="flex-1">
                          <p className="font-semibold text-slate-800">{goal.label}</p>
                          <p className="text-sm text-slate-500">{goal.desc}</p>
                        </div>
                        {profile.dietary_goal === goal.id && (
                          <Check className="w-5 h-5 text-emerald-500" />
                        )}
                      </div>
                    </motion.button>
                  ))}
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 1 && (
            <SlideTransition key="diet">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Dietary preference?</h1>
                  <p className="text-slate-500 mt-1">Select your eating style</p>
                </div>
                <div className="grid grid-cols-2 gap-3">
                  {dietTypes.map(diet => (
                    <motion.button
                      key={diet.id}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile(p => ({ ...p, diet_type: diet.id }))}
                      className={`p-4 rounded-2xl border-2 text-center transition-all ${
                        profile.diet_type === diet.id
                          ? 'border-emerald-500 bg-emerald-50'
                          : 'border-slate-200 bg-white hover:border-slate-300'
                      }`}
                    >
                      <span className="text-2xl block mb-1">{diet.emoji}</span>
                      <p className="font-medium text-slate-800 text-sm">{diet.label}</p>
                    </motion.button>
                  ))}
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 2 && (
            <SlideTransition key="stats">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Your stats</h1>
                  <p className="text-slate-500 mt-1">Help us calculate your targets</p>
                </div>
                <div className="space-y-4">
                  <div className="bg-white rounded-2xl p-4 border border-slate-200">
                    <div className="flex items-center gap-3 mb-2">
                      <Scale className="w-5 h-5 text-slate-400" />
                      <label className="font-medium text-slate-700">Weight (kg)</label>
                    </div>
                    <Input
                      type="number"
                      placeholder="70"
                      value={profile.weight}
                      onChange={(e) => setProfile(p => ({ ...p, weight: e.target.value }))}
                      className="border-0 text-xl font-semibold p-0 h-auto focus-visible:ring-0"
                    />
                  </div>
                  <div className="bg-white rounded-2xl p-4 border border-slate-200">
                    <div className="flex items-center gap-3 mb-2">
                      <Ruler className="w-5 h-5 text-slate-400" />
                      <label className="font-medium text-slate-700">Height (cm)</label>
                    </div>
                    <Input
                      type="number"
                      placeholder="170"
                      value={profile.height}
                      onChange={(e) => setProfile(p => ({ ...p, height: e.target.value }))}
                      className="border-0 text-xl font-semibold p-0 h-auto focus-visible:ring-0"
                    />
                  </div>
                  <div className="bg-white rounded-2xl p-4 border border-slate-200">
                    <div className="flex items-center gap-3 mb-2">
                      <Target className="w-5 h-5 text-slate-400" />
                      <label className="font-medium text-slate-700">Age</label>
                    </div>
                    <Input
                      type="number"
                      placeholder="30"
                      value={profile.age}
                      onChange={(e) => setProfile(p => ({ ...p, age: e.target.value }))}
                      className="border-0 text-xl font-semibold p-0 h-auto focus-visible:ring-0"
                    />
                  </div>
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 3 && (
            <SlideTransition key="activity">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Activity level</h1>
                  <p className="text-slate-500 mt-1">How active are you?</p>
                </div>
                <div className="space-y-3">
                  {activityLevels.map(level => (
                    <motion.button
                      key={level.id}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile(p => ({ ...p, activity_level: level.id }))}
                      className={`w-full p-4 rounded-2xl border-2 text-left transition-all ${
                        profile.activity_level === level.id
                          ? 'border-emerald-500 bg-emerald-50'
                          : 'border-slate-200 bg-white hover:border-slate-300'
                      }`}
                    >
                      <div className="flex items-center justify-between">
                        <div>
                          <p className="font-semibold text-slate-800">{level.label}</p>
                          <p className="text-sm text-slate-500">{level.desc}</p>
                        </div>
                        {profile.activity_level === level.id && (
                          <Check className="w-5 h-5 text-emerald-500" />
                        )}
                      </div>
                    </motion.button>
                  ))}
                </div>
              </div>
            </SlideTransition>
          )}

          {step === 4 && (
            <SlideTransition key="allergies">
              <div className="space-y-6">
                <div>
                  <h1 className="text-2xl font-bold text-slate-800">Any allergies?</h1>
                  <p className="text-slate-500 mt-1">We'll alert you when scanning</p>
                </div>
                <div className="flex flex-wrap gap-2">
                  {commonAllergens.map(allergen => (
                    <Badge
                      key={allergen}
                      variant={profile.allergies.includes(allergen) ? 'default' : 'outline'}
                      className={`px-4 py-2 text-sm cursor-pointer transition-all ${
                        profile.allergies.includes(allergen)
                          ? 'bg-red-500 hover:bg-red-600 border-red-500'
                          : 'hover:border-slate-400'
                      }`}
                      onClick={() => toggleAllergen(allergen)}
                    >
                      {profile.allergies.includes(allergen) && (
                        <AlertCircle className="w-3 h-3 mr-1" />
                      )}
                      {allergen}
                    </Badge>
                  ))}
                </div>
                <p className="text-sm text-slate-400">Tap to select, skip if none</p>
              </div>
            </SlideTransition>
          )}
        </AnimatePresence>
      </div>

      {/* Navigation */}
      <div className="p-6 bg-white border-t border-slate-100">
        <div className="flex gap-3">
          {step > 0 && (
            <Button
              variant="outline"
              onClick={() => setStep(s => s - 1)}
              className="h-14 px-6 rounded-2xl"
            >
              <ChevronLeft className="w-5 h-5" />
            </Button>
          )}
          <Button
            onClick={() => step < totalSteps - 1 ? setStep(s => s + 1) : handleComplete()}
            disabled={!canProceed()}
            className="flex-1 h-14 rounded-2xl text-base bg-gradient-to-r from-emerald-500 to-teal-600 hover:from-emerald-600 hover:to-teal-700"
          >
            {step < totalSteps - 1 ? (
              <>
                Continue
                <ChevronRight className="w-5 h-5 ml-2" />
              </>
            ) : (
              <>
                <Sparkles className="w-5 h-5 mr-2" />
                Start Scanning
              </>
            )}
          </Button>
        </div>
      </div>
    </div>
  );
}